In [7]:
!pip install --upgrade --quiet langchain langchain-community langchain-groq neo4j

In [8]:
## Graphdb configuration
NEO4J_URI = "neo4j+s://databases.neo4j.io"
NEO4J_USERNAME = "neo4asdTharunj"
NEO4J_PASSWORD = "Zr3pzkqiCHwasddADAl7mfYIJplzQNDxLYHyAbUjgnk5tmkBus"

In [9]:
import os
os.environ["NEO4J_URI"] = NEO4J_URI
os.environ["NEO4J_USERNAME"] = NEO4J_USERNAME
os.environ["NEO4J_PASSWORD"] = NEO4J_PASSWORD


In [10]:
from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
)

In [11]:
graph

In [22]:
import warnings
import logging
import os
import sys
from typing import List, Tuple
import pandas as pd
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.evaluation import load_evaluator
from datasets import load_dataset
import transformers
from tqdm import tqdm
import shutil
from dotenv import load_dotenv
import gc
import torch
from langchain.embeddings import OpenAIEmbeddings
from langchain_core.documents import Document
# Suppress warnings and set logging levels
warnings.filterwarnings('ignore')
logging.getLogger().setLevel(logging.ERROR)
transformers.logging.set_verbosity_error()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

In [13]:
!pip install --upgrade --quiet python-dotenv

In [14]:
!pip install --upgrade --quiet transformers

In [15]:
!pip install --upgrade --quiet datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [23]:
import os
import shutil
# Remove existing data
if os.path.exists('data'):
    shutil.rmtree('data')
os.makedirs('data', exist_ok=True)

from dotenv import load_dotenv
# Load environment variables
load_dotenv()

OPENAI_API_KEY="sk-proj-Hs27NKk1j3J5bzwBZESpcwasdasdasaSk-0gcEZcTK9qwnG1VvoYzLhaihzu6-JFXA2VSgI3PJpxm6PrE7T3BlbkFJ3DCCAyxG4bPC3S-MxWT0gdWMlwe68-GrC7gX9k_oJLozVHBSla6lY_dBGWax_qbKVYjT54asdasdasdDfoA"
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="lsv2_pt_7e6bf388f07146fbbf8e56d5057asdasdasddasasdas63e07_7143f4f497"
LANGCHAIN_PROJECT="langchain_tutorial"


os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["LANGCHAIN_ENDPOINT"] = LANGCHAIN_ENDPOINT
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY


In [24]:

# Constants for tiny dataset
MAX_PASSAGES = 100  # Extremely small dataset
MAX_QA_PAIRS = 20   # Very few QA pairs
BATCH_SIZE = 2      # Process just 2 documents at a time

# Remove existing data
if os.path.exists('data'):
    shutil.rmtree('data')
os.makedirs('data', exist_ok=True)




In [25]:
# Constants for tiny dataset
MAX_PASSAGES = 100  # Extremely small dataset
MAX_QA_PAIRS = 20   # Very few QA pairs
BATCH_SIZE = 2      # Process just 2 documents at a time

# Remove existing data and create new directory
if os.path.exists('data'):
    shutil.rmtree('data')
os.makedirs('data', exist_ok=True)


embed_model = None
vectorstore = None
chain = None
passages = None
test_data = None
persist_directory = "data/chroma_db"

# Create persist directory if it doesn't exist
if not os.path.exists(persist_directory):
    os.makedirs(persist_directory)

In [26]:
def initialize_models(model_name: str = "gpt-3.5-turbo", temperature: float = 0):
    """Initialize the embedding model, LLM, and prompt template"""
    global embed_model, chain

    # Initialize embedding model

    embed_model = OpenAIEmbeddings(
      model="text-embedding-3-small"
    )

    # Initialize LLM
    llm = ChatOpenAI(model_name=model_name, temperature=temperature)

    # Initialize prompt template
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You are a helpful biomedical research assistant. Answer briefly."),
        ("user", "Context: {context}\n\nQuestion: {question}")
    ])

    return embed_model, llm, prompt

# Run initialization
embed_model, llm, prompt = initialize_models()

In [27]:
def download_and_save_dataset():
    """Download and process BioASQ dataset"""
    try:
        # Load datasets
        passages_dataset = load_dataset("rag-datasets/rag-mini-bioasq", 'text-corpus')
        qa_dataset = load_dataset("rag-datasets/rag-mini-bioasq", 'question-answer-passages')

        # Get correct splits
        passages_split = next(iter(passages_dataset.keys()))
        qa_split = next(iter(qa_dataset.keys()))

        # Sample tiny subsets
        passages_df = pd.DataFrame(passages_dataset[passages_split]).sample(
            n=min(MAX_PASSAGES, len(passages_dataset[passages_split])),
            random_state=42
        )
        qa_df = pd.DataFrame(qa_dataset[qa_split]).sample(
            n=min(MAX_QA_PAIRS, len(qa_dataset[qa_split])),
            random_state=42
        )

        # Filter out invalid passages and convert to Documents
        passages_df = passages_df.dropna(subset=['passage'])
        documents = [
            Document(
                page_content=row['passage'],
                metadata={'id': row['id']}
            )
            for _, row in passages_df.iterrows()
            if isinstance(row['passage'], str)
        ]

        # Save to disk
        passages_df.to_parquet('data/passages.parquet')
        qa_df.to_parquet('data/qa_pairs.parquet')

        return documents, passages_df, qa_df

    except Exception as e:
        print(f"Error downloading dataset: {str(e)}")
        raise

# Load the dataset
documents, passages_df, qa_pairs_df = download_and_save_dataset()

README.md:   0%|          | 0.00/813 [00:00<?, ?B/s]

part.0.parquet:   0%|          | 0.00/24.5M [00:00<?, ?B/s]

Generating passages split:   0%|          | 0/40221 [00:00<?, ? examples/s]

part.0.parquet:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

Generating test split:   0%|          | 0/4719 [00:00<?, ? examples/s]

In [28]:
Documents = download_and_save_dataset()
docs,passages,qa_pairs = Documents
passages.head()


,passage,id
35949,CRISPR-Cas12a (Cpf1) proteins are RNA-guided e...,29449511
29691,nan,25542770
22896,nan,23180957
7096,The objective of this prospective study was th...,15157997
857,nan,2724231


In [29]:
qa_pairs.head()


,question,answer,relevant_passage_ids,id
1210,What are the effects of homozygosity of EDNRB ...,Three susceptibility genes have been recently ...,"[9885824, 11891690, 12939697, 8852660, 1623755...",1210
2029,What is the function of calcium-sensing recept...,The calcium-sensing receptor (CaSR) is a G-pro...,"[26608608, 23856267, 26261299, 26386835, 21566...",2029
3979,What are commensal bacteria?,The gut microbiota is composed of a large numb...,"[30672882, 31049923, 32758517, 33186243]",3979
3729,What are the eRNA-producing centers (EPCs)?,Active enhancers in mammals produce enhancer R...,[30447999],3729
3704,Does radiotherapy for prostate cancer increase...,"Yes, radiotherapy for prostate cancer is assoc...","[27844130, 27432454, 28499661, 18801517, 25900...",3704


In [30]:
!pip install --upgrade --quiet langchain-experimental

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.0/209.0 kB 1.5 MB/s eta 0:00:00


In [31]:
models= initialize_models()
embed_model, llm, prompt = models

In [34]:
!pip install json-repair

In [35]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)

In [36]:
def create_knowledge_graph(documents):
    """Create knowledge graph with proper property handling"""
    try:
        # Clear existing graph
        print("Clearing existing graph...")
        graph.query("MATCH (n) DETACH DELETE n")

        # Convert documents to graph format
        print("Converting documents to graph format...")
        graph_docs = llm_transformer.convert_to_graph_documents(documents)

        # First pass: Create nodes
        print("\nCreating nodes...")
        node_ids = set()  # Track created nodes

        for doc in graph_docs:
            for node in doc.nodes:
                try:
                    # Sanitize node type and ID
                    node_type = node.type.replace(' ', '_').replace('-', '_').replace('/', '_').upper()
                    node_id = str(node.id).replace(' ', '_').replace('-', '_')

                    # Store only primitive values
                    props = {
                        'id': node_id,
                        'name': str(node.id),
                        'type': node_type
                    }

                    # Add any additional primitive properties
                    if node.properties:
                        for k, v in node.properties.items():
                            if isinstance(v, (str, int, float, bool)):
                                props[k] = v

                    # Create node
                    cypher = """
                    MERGE (n:%s {id: $props.id})
                    SET n = $props
                    """ % node_type

                    graph.query(cypher, {'props': props})
                    node_ids.add(node_id)
                    print(f"Created node: {node_type} - {node_id}")

                except Exception as e:
                    print(f"Error creating node {node.id}: {str(e)}")
                    print(f"Node properties: {props}")

        # Second pass: Create relationships
        print("\nCreating relationships...")
        rels_created = 0

        for doc in graph_docs:
            for rel in doc.relationships:
                try:
                    # Sanitize IDs
                    source_id = str(rel.source.id).replace(' ', '_').replace('-', '_')
                    target_id = str(rel.target.id).replace(' ', '_').replace('-', '_')

                    # Only create relationship if both nodes exist
                    if source_id in node_ids and target_id in node_ids:
                        # Sanitize relationship type
                        rel_type = rel.type.replace(' ', '_').replace('-', '_').replace('/', '_').upper()

                        cypher = """
                        MATCH (source {id: $source_id})
                        MATCH (target {id: $target_id})
                        MERGE (source)-[r:%s]->(target)
                        """ % rel_type

                        graph.query(cypher, {
                            'source_id': source_id,
                            'target_id': target_id
                        })

                        rels_created += 1
                        print(f"Created relationship {rels_created}: {source_id} -[{rel_type}]-> {target_id}")

                except Exception as e:
                    print(f"Error creating relationship: {str(e)}")

        # Verify the graph
        node_count = graph.query("MATCH (n) RETURN count(n) as count")[0]['count']
        rel_count = graph.query("MATCH ()-[r]->() RETURN count(r) as count")[0]['count']

        print(f"\nFinal graph statistics:")
        print(f"Nodes created: {len(node_ids)}")
        print(f"Nodes in database: {node_count}")
        print(f"Relationships created: {rels_created}")
        print(f"Relationships in database: {rel_count}")

        # Print sample of created graph
        print("\nSample of graph contents:")
        sample = graph.query("""
        MATCH (n)-[r]->(m)
        RETURN labels(n)[0] as source_type,
               n.id as source_id,
               type(r) as relationship,
               labels(m)[0] as target_type,
               m.id as target_id
        LIMIT 5
        """)

        for row in sample:
            print(f"({row['source_type']}){row['source_id']} -[{row['relationship']}]-> ({row['target_type']}){row['target_id']}")

    except Exception as e:
        print(f"Error in graph creation: {str(e)}")
        raise

# Add verification function
def verify_graph():
    """Verify graph contents"""
    print("\nVerifying graph contents:")

    # Check node types and counts
    print("\nNode types and counts:")
    node_counts = graph.query("""
    MATCH (n)
    WITH labels(n)[0] as node_type, count(n) as count
    RETURN node_type, count
    ORDER BY count DESC
    """)

    for row in node_counts:
        print(f"{row['node_type']}: {row['count']} nodes")

    # Check relationship types and counts
    print("\nRelationship types and counts:")
    rel_counts = graph.query("""
    MATCH ()-[r]->()
    WITH type(r) as rel_type, count(r) as count
    RETURN rel_type, count
    ORDER BY count DESC
    """)

    for row in rel_counts:
        print(f"{row['rel_type']}: {row['count']} relationships")

    # Check for disconnected nodes
    disconnected = graph.query("""
    MATCH (n)
    WHERE NOT (n)-[]-()
    RETURN count(n) as count
    """)[0]['count']

    print(f"\nDisconnected nodes: {disconnected}")

# Create and verify graph
create_knowledge_graph(docs)
verify_graph()

Clearing existing graph...
Converting documents to graph format...

Creating nodes...
Created node: PROPERTY - ssDNase_activation
Created node: ACTIVITY - DNA_detection
Created node: METHOD - DETECTR
Created node: SYSTEM - bacterial_adaptive_immune_systems
Created node: CHARACTERISTIC - generate_targeted,_double_stranded_DNA_breaks
Created node: PLATFORM - a_simple_platform_for_molecular_diagnostics
Created node: PROTEIN - Cas12a
Created node: ACTIVITY - rapid_and_specific_detection_of_human_papillomavirus_in_patient_samples
Created node: PROTEIN - CRISPR_Cas12a
Created node: ACTIVITY - indiscriminate_single_stranded_DNA_(ssDNA)_cleavage_activity
Created node: PROPERTY - nonspecific_single_stranded_deoxyribonuclease_(ssDNase)_cleavage
Created node: MARKER - additional_marker_of_SD_severity
Created node: MARKER - marker_of_mental_retardation
Created node: PATIENT - Patients_with_SD
Created node: ANATOMICAL_STRUCTURE - left_basal_ganglia
Created node: CHARACTERISTIC - reduced_ratios
Crea

In [38]:
!pip install chromadb --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 4.7 MB/s eta 0:00:00


In [39]:
!pip install tiktoken --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.6 MB/s eta 0:00:00


In [40]:
def create_vector_store(documents):
    """Create vector store for semantic search"""
    try:
        vectorstore = Chroma.from_documents(
            documents=documents,
            embedding=embed_model,
            persist_directory=persist_directory
        )
        return vectorstore
    except Exception as e:
        print(f"Error creating vector store: {str(e)}")
        raise

# Create vector store
vectorstore = create_vector_store(documents)

## GraphRag Retriever


In [47]:
from typing import List, Dict,Any
class GraphRAGRetriever:
    def __init__(self, graph, vectorstore, llm):
        """Initialize GraphRAG retriever with both graph and vector components"""
        self.graph = graph
        self.vectorstore = vectorstore
        self.llm = llm

    def get_graph_context(self, query: str, k: int = 3) -> List[Dict]:
        """Get relevant context from knowledge graph"""
        try:
            # Query for relevant nodes and their relationships
            cypher = """
            WITH split($query, ' ') as words
            UNWIND words as word
            MATCH (n)-[r]->(m)
            WHERE any(w in words WHERE
                  toLower(n.name) CONTAINS toLower(w) OR
                  toLower(m.name) CONTAINS toLower(w))
            RETURN DISTINCT n.name as source,
                   type(r) as relation,
                   m.name as target
            LIMIT $limit
            """

            results = self.graph.query(cypher, {
                "query": query,
                "limit": k
            })
            return results

        except Exception as e:
            print(f"Error querying graph: {str(e)}")
            return []

    def get_vector_context(self, query: str, k: int = 3) -> List[Document]:
        """Get relevant context from vector store"""
        try:
            return self.vectorstore.similarity_search(query, k=k)
        except Exception as e:
            print(f"Error querying vector store: {str(e)}")
            return []

    def combine_contexts(self, graph_results: List[Dict], vector_results: List[Document]) -> str:
        """Combine graph and vector evidence into unified context"""
        combined_context = []

        # Add graph relationships
        if graph_results:
            combined_context.append("Graph Knowledge:")
            for result in graph_results:
                relationship = f"{result['source']} {result['relation']} {result['target']}"
                combined_context.append(f"- {relationship}")

        # Add vector store results
        if vector_results:
            combined_context.append("\nDetailed Context:")
            for i, doc in enumerate(vector_results, 1):
                combined_context.append(f"[{i}] {doc.page_content}")

        return "\n".join(combined_context)

    def get_relevant_context(self, query: str, k: int = 3) -> str:
        """Get combined relevant context from both sources"""
        # Get graph context
        graph_results = self.get_graph_context(query, k)

        # Get vector context
        vector_results = self.get_vector_context(query, k)

        # Combine contexts
        return self.combine_contexts(graph_results, vector_results)

In [61]:
class GraphRAG:
    def __init__(self, graph, vectorstore, llm):
        self.retriever = GraphRAGRetriever(graph, vectorstore, llm)
        self.llm = llm
        self.qa_pairs = qa_pairs_df

        # Initialize primary prompt template
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", """You are a biomedical research assistant. Use both structured knowledge
            and detailed context to provide comprehensive answers."""),
            ("user", """Context: {context}
            Question: {question}
            Provide a comprehensive answer.""")
        ])

        # Initialize evaluation prompt
        self.eval_prompt = ChatPromptTemplate.from_messages([
            ("system", """You are an evaluation expert. Evaluate the generated answer against the ground truth.
            Provide numerical scores in the exact format:
            ACCURACY: [score between 0-1]
            COMPLETENESS: [score between 0-1]
            RELEVANCE: [score between 0-1]
            EXPLANATION: [your explanation]"""),
            ("user", """Question: {question}
            Generated Answer: {generated}
            Ground Truth: {ground_truth}

            Provide evaluation scores and explanation:""")
        ])

    def query(self, question: str) -> Dict[str, Any]:
        """Process a query through the GraphRAG system"""
        try:
            # Get context from retriever
            context = self.retriever.get_relevant_context(question)

            # Generate response using LLM
            chain = (
                self.prompt
                | self.llm
                | StrOutputParser()
            )

            response = chain.invoke({
                "context": context,
                "question": question
            })

            return {
                "question": question,
                "answer": response,
                "context_used": context
            }

        except Exception as e:
            print(f"Error in query processing: {str(e)}")
            return {
                "question": question,
                "answer": "Error: Could not generate answer.",
                "context_used": ""
            }

    def evaluate_answer(self, question: str, generated_answer: str, ground_truth: str) -> Dict:
        """Evaluate a generated answer against ground truth"""
        try:
            chain = (
                self.eval_prompt
                | self.llm
                | StrOutputParser()
            )

            eval_response = chain.invoke({
                "question": question,
                "generated": generated_answer,
                "ground_truth": ground_truth
            })

            # Initialize scores
            scores = {
                "accuracy": 0.0,
                "completeness": 0.0,
                "relevance": 0.0,
                "explanation": ""
            }

            # Parse evaluation response
            lines = eval_response.strip().split('\n')
            for line in lines:
                line = line.strip().upper()
                if line.startswith('ACCURACY:'):
                    try:
                        scores["accuracy"] = float(line.split(':')[1].strip())
                    except:
                        print(f"Error parsing accuracy: {line}")
                elif line.startswith('COMPLETENESS:'):
                    try:
                        scores["completeness"] = float(line.split(':')[1].strip())
                    except:
                        print(f"Error parsing completeness: {line}")
                elif line.startswith('RELEVANCE:'):
                    try:
                        scores["relevance"] = float(line.split(':')[1].strip())
                    except:
                        print(f"Error parsing relevance: {line}")
                elif line.startswith('EXPLANATION:'):
                    scores["explanation"] = line.split(':')[1].strip()

            return scores

        except Exception as e:
            print(f"Error in evaluation: {str(e)}")
            return {
                "accuracy": 0.0,
                "completeness": 0.0,
                "relevance": 0.0,
                "explanation": f"Evaluation error: {str(e)}"
            }

    def run_evaluation(self, num_samples: int = 5) -> Dict:
        """Run evaluation on sample questions"""
        eval_results = []

        try:
            # Sample questions
            sample_qa = self.qa_pairs.sample(n=min(num_samples, len(self.qa_pairs)))

            for _, row in sample_qa.iterrows():
                try:
                    question = row['question']
                    ground_truth = row['answer']

                    print(f"\nEvaluating question: {question}")

                    # Get system response
                    result = self.query(question)
                    generated_answer = result['answer']

                    print("\nGenerated Answer:", generated_answer)
                    print("Ground Truth:", ground_truth)

                    # Evaluate answer
                    evaluation = self.evaluate_answer(
                        question=question,
                        generated_answer=generated_answer,
                        ground_truth=ground_truth
                    )

                    eval_results.append({
                        "question": question,
                        "generated_answer": generated_answer,
                        "ground_truth": ground_truth,
                        "evaluation": evaluation,
                        "context_used": result['context_used']
                    })

                    # Print scores
                    print("\nScores:")
                    print(f"Accuracy: {evaluation['accuracy']:.2f}")
                    print(f"Completeness: {evaluation['completeness']:.2f}")
                    print(f"Relevance: {evaluation['relevance']:.2f}")
                    if evaluation['explanation']:
                        print("\nExplanation:", evaluation['explanation'])

                except Exception as e:
                    print(f"Error evaluating question: {str(e)}")
                    continue

            # Calculate average scores
            if eval_results:
                avg_scores = {
                    "accuracy": sum(r["evaluation"]["accuracy"] for r in eval_results) / len(eval_results),
                    "completeness": sum(r["evaluation"]["completeness"] for r in eval_results) / len(eval_results),
                    "relevance": sum(r["evaluation"]["relevance"] for r in eval_results) / len(eval_results)
                }
            else:
                avg_scores = {"accuracy": 0.0, "completeness": 0.0, "relevance": 0.0}

            return {
                "detailed_results": eval_results,
                "average_scores": avg_scores
            }

        except Exception as e:
            print(f"Error in evaluation process: {str(e)}")
            return {
                "detailed_results": [],
                "average_scores": {"accuracy": 0.0, "completeness": 0.0, "relevance": 0.0}
            }

In [62]:
def interactive_graphrag(graph_rag: GraphRAG):
    """Interactive interface for GraphRAG system"""
    while True:
        print("\n" + "="*50)
        print("GraphRAG Query Interface")
        print("="*50)
        print("1. Ask a question")
        print("2. View sample questions")
        print("3. Run evaluation")
        print("4. Exit")

        choice = input("\nEnter your choice (1-4): ")

        if choice == "1":
            question = input("\nEnter your question: ")
            print("\nProcessing query...")

            result = graph_rag.query(question)

            print("\nAnswer:", result["answer"])
            print("\nContext Used:")
            print(result["context_used"])

            feedback = input("\nWas this answer helpful? (y/n): ")
            if feedback.lower() == 'n':
                print("Please provide specific feedback to help improve the system:")
                feedback = input()

        elif choice == "2":
            print("\nSample Questions:")
            sample_questions = graph_rag.qa_pairs['question'].sample(n=5).tolist()
            for i, q in enumerate(sample_questions, 1):
                print(f"{i}. {q}")

            sample_choice = input("\nSelect a question number or press Enter to continue: ")
            if sample_choice.isdigit() and 1 <= int(sample_choice) <= len(sample_questions):
                result = graph_rag.query(sample_questions[int(sample_choice)-1])
                print("\nAnswer:", result["answer"])
                print("\nContext Used:")
                print(result["context_used"])

        elif choice == "3":
            num_samples = input("\nEnter number of samples to evaluate (default: 5): ")
            try:
                num_samples = int(num_samples)
            except ValueError:
                num_samples = 5

            print(f"\nRunning evaluation with {num_samples} samples...")
            eval_results = graph_rag.run_evaluation(num_samples)

            print("\nEvaluation Results:")
            print("Average Scores:")
            print(f"Accuracy: {eval_results['average_scores']['accuracy']:.2f}")
            print(f"Completeness: {eval_results['average_scores']['completeness']:.2f}")
            print(f"Relevance: {eval_results['average_scores']['relevance']:.2f}")

            # Ask if user wants to see detailed results
            see_details = input("\nWould you like to see detailed results? (y/n): ")
            if see_details.lower() == 'y':
                for result in eval_results['detailed_results']:
                    print("\n" + "="*50)
                    print("Question:", result['question'])
                    print("\nGenerated:", result['generated_answer'])
                    print("\nGround Truth:", result['ground_truth'])
                    print("\nScores:")
                    print(f"Accuracy: {result['evaluation']['accuracy']:.2f}")
                    print(f"Completeness: {result['evaluation']['completeness']:.2f}")
                    print(f"Relevance: {result['evaluation']['relevance']:.2f}")

        elif choice == "4":
            print("\nExiting GraphRAG system...")
            break

        else:
            print("\nInvalid choice. Please try again.")



In [63]:
# Initialize and test
graph_rag = GraphRAG(graph, vectorstore, llm)

# Test single evaluation
question = "What is CRISPR-Cas12a?"
result = graph_rag.query(question)
evaluation = graph_rag.evaluate_answer(
    question=question,
    generated_answer=result['answer'],
    ground_truth="CRISPR-Cas12a is a RNA-guided enzyme..."
)
print("\nTest evaluation:", evaluation)

# Or run full evaluation
results = graph_rag.run_evaluation(num_samples=3)
print("\nEvaluation results:", results)


Test evaluation: {'accuracy': 1.0, 'completeness': 0.9, 'relevance': 1.0, 'explanation': 'THE GENERATED ANSWER IS HIGHLY ACCURATE, PROVIDING DETAILED INFORMATION ABOUT CRISPR-CAS12A, INCLUDING ITS ROLE IN GENOME EDITING AND MOLECULAR DIAGNOSTICS. IT COVERS THE UNIQUE FEATURES AND APPLICATIONS OF CRISPR-CAS12A EFFECTIVELY. THE COMPLETENESS SCORE IS SLIGHTLY LOWER DUE TO SOME ADDITIONAL TECHNICAL DETAILS THAT COULD HAVE BEEN INCLUDED, BUT OVERALL THE ANSWER IS COMPREHENSIVE. THE RELEVANCE SCORE IS HIGH AS THE INFORMATION PROVIDED DIRECTLY ADDRESSES THE QUESTION ABOUT CRISPR-CAS12A.'}

Evaluating question: What is the mode of action of Hsp90 inhibitors?

Generated Answer: Heat shock protein 90 (Hsp90) inhibitors are a class of compounds that target the molecular chaperone Hsp90, which plays a crucial role in protein folding, stability, and function. Hsp90 is involved in the folding and maturation of a wide range of client proteins, many of which are key regulators of cell growth, surviva

In [64]:
# Usage example:
def setup_and_run_graphrag():
    """Setup and run GraphRAG system"""
    # Initialize GraphRAG
    graph_rag = GraphRAG(graph, vectorstore, llm)

    # Run interactive interface
    interactive_graphrag(graph_rag)

# Run the system
if __name__ == "__main__":
      setup_and_run_graphrag()


GraphRAG Query Interface
1. Ask a question
2. View sample questions
3. Run evaluation
4. Exit

Enter your choice (1-4): 3

Enter number of samples to evaluate (default: 5): 2

Running evaluation with 2 samples...

Evaluating question: Which is the specificity of deubiquitinase USP25?

Generated Answer: Deubiquitinase USP25 is a protein that belongs to the ubiquitin-specific protease (USP) family, which plays a crucial role in the regulation of protein degradation and turnover by removing ubiquitin molecules from target proteins. The specificity of deubiquitinase USP25 refers to its ability to selectively recognize and cleave specific ubiquitin chains or substrates, thereby influencing various cellular processes.

1. **Substrate Specificity**: USP25 exhibits substrate specificity by recognizing and deubiquitinating specific target proteins. This specificity can be determined by the sequence or structure of the ubiquitin chains attached to the substrate proteins. USP25 may preferentiall